## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

<h3> Разделим нашу работу на несколько этапов: </h3>

* Получение общей информации из исходного файла
* Предобработка данных:

    *-Обработка пропущенных значений*
    
    *-Обработка повторящихся значений*
    
    *-Лемматизация*
    
    *-Категоризация*
    
* Ответы на поставленные вопросы:

    *-Есть ли зависимость между наличием детей и возвратом кредита в срок?*
    
    *-Есть ли зависимость между семейным положением и возвратом кредита в срок?*
    
    *-Есть ли зависимость между уровнем дохода и возвратом кредита в срок?*
    
    *-Как разные цели кредита влияют на его возврат в срок?*
    
* Общий вывод из полученных результатов

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [40]:
import pandas as pd
from pymystem3 import Mystem

In [41]:
df = pd.read_csv('/datasets/data.csv')

In [42]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Рассмотрим полученную таблицу *df*

Всего в таблице 12 столбцов:
* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита

In [44]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Обратим внимание на некорректные значения в столбце *days_empolyed*. В столбце *children* присутствуют отрицательные значения, а в столбце *dob_years* нулевые значения.

### Вывод

В каждой строке содержится информация о человеке, который взял кредит в банке. Для ответов на поставленные вопросы особенно важен столбец *debt*, показывающий отдал человек кредит или нет.

Обратим внимание на некорректные данные в столбце 'days_employed'. Можем заметить, что значения в этом столбце много больше нуля для пенсионеров, а для остальных меньше. Преполагаем, что это техническая ошибка связанная с заполнением таблицы в разные периоды времени. Также бросаются в глаза разные регистры в столбце education

### Шаг 2. Предобработка данных

### Обработка пропусков

In [45]:
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Обнаружили 2174 пропусков с столбцах days_employed и total_income

In [46]:
df[(df['days_employed'].isnull()) & (df['total_income'].isnull())].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


### Вывод

Значения пропущены в столбцах 'days_employed' и 'total_income' по предположению ввиду отстутвия данных в источнике, причём зависимость прямая, т.е. если значение пропущено в 'days_employed', то оно пропущено и в 'total_income'.

Т.к. напрямую столбец days_empolyed в нашей задаче ни на что не влияет, то заменим пропущенные значение на 0. Пропущенные значение в стоблце total_income заменим на отрицательное число. В момент, когда понадобится этот столбец применим фильтрацию положительным значениям. 

In [47]:
df['days_employed'] = df['days_employed'].fillna(0)
df['total_income'] = df['total_income'].fillna(-1)
df.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Действительно, пустых значений в таблице больше нет.

### Замена типа данных

In [48]:
df['days_employed'] = df['days_employed'].astype(int)
df['total_income'] = df['total_income'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

Для дальнейшей работы заменили столбцы типа Float на Int методом astype(). Отличие от to_numeric() лишь в количестве входных параметров, конструкция astype() выглядит приятнее для использования.

### Обработка дубликатов

In [49]:
df.duplicated().sum()

54

Обнаружили 54 дубликата в исходной таблице. Удалим обнаруженные повторения.

In [50]:
df = df.drop_duplicates().reset_index(drop=True)

In [51]:
df.duplicated().sum()

0

In [52]:
df['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Заметим разные регистры в столбце 'education' для одних и тех же значений.
Исправим ситуацию.

In [53]:
df['education'] = df['education'].str.lower()
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Теперь с регистрами всё в порядке.

In [54]:
df.duplicated().sum()

17

С учётом регистра получили еще 17 дупликатов. Аналогично удалим их из таблицы.

In [55]:
df = df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum()

0

### Вывод

Наличие дубликатов в исходной таблице говорит о некой технической ошибке. Также в процессе обнаружились дупилкаты по разным регистрам, что скорее всего говорит о человеческом факторе.

### Лемматизация

Для удобства работы со столбцом purpose приведем каждое слово к своей лемме. Т.е. проведем лемматизацию, что позволит
в дальнейшем категоризовать наши данные по основным целям. Для этого подключим библиотеку pymystem3

In [56]:
m = Mystem()
 
def lem(row):
    return m.lemmatize(row['purpose']) 
 
df['purpose_lem'] = df.apply(lem, axis=1)
df['purpose_lem'].head(10)

0                      [покупка,  , жилье, \n]
1            [приобретение,  , автомобиль, \n]
2                      [покупка,  , жилье, \n]
3         [дополнительный,  , образование, \n]
4                    [сыграть,  , свадьба, \n]
5                      [покупка,  , жилье, \n]
6               [операция,  , с,  , жилье, \n]
7                            [образование, \n]
8          [на,  , проведение,  , свадьба, \n]
9    [покупка,  , жилье,  , для,  , семья, \n]
Name: purpose_lem, dtype: object

### Вывод

Лемматизировали стобец purpose и записали полученную лемматизированную строку в новый столбец purpose_lem

### Категоризация данных

In [57]:
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Из списка уникальных значений выделим следующие категории: Недвижимость, автомобиль, образование, свадьба

In [58]:
def purpose_group(row):
    if ('жилье' in row) or ('недвижимость' in row):
        return 'недвижимость'
    elif 'свадьба' in row:
        return 'свадьба'
    elif 'образование' in row:
        return 'образование'
    elif 'автомобиль' in row:
        return 'автомобиль'
    
df['purpose_group'] = df['purpose_lem'].apply(purpose_group)
df['purpose_group'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_group, dtype: int64

Действительно, эти категории покрыли все значения столбца purpose

### Вывод

Категоризовали таблицу по столбцу purpose. Выделили 4 основные категории: Недвижимость, автомобиль, образование, свадьба.
Можем заметить, что количество обращений связаных с недвижимостью первосходит сумму всех остальных.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Создадим отдельную таблицу для удобства работы с колонками children и debt

In [59]:
debt_child = df[['children', 'debt']]
debt_child['children'].unique()
debt_child_group = debt_child.groupby('children').count()
debt_child_group

,debt
children,
-1,47
0,14091
1,4808
2,2052
3,330
4,41
5,9
20,76


Обратим внимание на некорректные данные: 47 строк со значением -1 в столбце children.
Не будем принимать в рассчёт эти строки.

Сгруппируем таблицу по количеству детей и подсчитаем количество взятых кредитов для каждой категории.

In [60]:
debt_child = debt_child[debt_child['children'] != -1]
child = debt_child.groupby(['children']).agg({'debt':'count'})
child['fail'] = debt_child[debt_child['debt']==1].groupby(['children']).agg({'debt':'count'})
try:
    child['percent'] = child['fail'] / child['debt'] *100
except:
    print('warning: check child[]')
child

,debt,fail,percent
children,,,
0,14091,1063.0,7.543822
1,4808,444.0,9.234609
2,2052,194.0,9.454191
3,330,27.0,8.181818
4,41,4.0,9.756098
5,9,NaN,NaN
20,76,8.0,10.526316


По построенной таблице видно, что люди без детей чаще погашают кредиты.
Обратим внимание на то, что среди людей с детьми, люди имеющие 3 ребёнка чаще выплачивают кредит. Информации о людях с 5 детьми много меньше, чем об остальных, поэтому этой строкой перенебрегаем для выводов.

### Вывод

Чаще всех выплачивают кредит в срок люди без детей, лишь 7.54% из них не выплатили кредит в срок.
Хуже всех дела у многодетных, а именно - 10.52% людей, имеющих 20 детей не выплатили кредит в срок, но количество таких людей не велико. Важнее рассмотреть более распространённый случай, а именно наличие 1 ребёнка. 9.23% людей, имеющих 1 ребёнка не выплатили кредит в срок.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Создадим отдельную таблицу для удобства работы со столбцами family_status и debt

In [61]:
debt_family = df[['family_status', 'debt']]
debt_family['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

Проверили уникальные значения в столбце family_satus. Всё в порядке.

Сгруппируем таблицу по семейному статусу и подсчитаем количество взятых кредитов и неоплаченных кредитов для каждой категории.

In [62]:
family = debt_family.groupby(['family_status']).agg({'debt':'count'})
family['fail'] = debt_family[debt_family['debt']==1].groupby(['family_status']).agg({'debt':'count'})
try:
    family['percent'] = family['fail'] / family['debt'] *100
except:
    print('warning: check family[]')
family.sort_values('percent')

,debt,fail,percent
family_status,,,
вдовец / вдова,959,63,6.569343
в разводе,1195,85,7.112971
женат / замужем,12339,931,7.545182
гражданский брак,4151,388,9.347145
Не женат / не замужем,2810,274,9.750890


### Вывод

Чаще остальных выплачивают кредит вовремя вдовцы и вдовы - всего 6.56% из них не выплатили его вовремя. Хуже всего ситуация у неженатых и незамужних, среди них не выплатили кредит вовремя 9.75%.
Среди самой большой группы, а именно женатые\замужние, не выплатили кредит вовремя 7.54%

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Создадим отдельную таблицу для удобства работы со столбцами total_income и debt

In [63]:
debt_income = df[['total_income', 'debt']]
debt_income = debt_income[debt_income['total_income']!=-1]
debt_income.head(10)

,total_income,debt
0,253875,0
1,112080,0
2,145885,0
3,267628,0
4,158616,0
5,255763,0
6,240525,0
7,135823,0
8,95856,0
9,144425,0


In [64]:
income_min = debt_income['total_income'].min()
income_max = debt_income['total_income'].max()
print('Минимальный доход: {}'.format(income_min))
print('Максимальный доход: {}'.format(income_max))

Минимальный доход: 20667
Максимальный доход: 2265604


Разобъём доход на 3 категории. Условно: низкий, средний, высокий по уровню дохода. Причём для более равномерного деления по классам воспользуемся принципом медианного деления.

In [65]:
delta = round(debt_income.shape[0] / 3)
debt_income = debt_income.sort_values('total_income').reset_index(drop=True)
income1 = debt_income.loc[delta]
income2 = debt_income.loc[2*delta]

Мы получили значения income1 и income2, которые разбивают доходы на 3 части. Теперь напишем функцию, определяющую в какую 
категорию входит доход каждого.

In [66]:
def income_group(income):
    if income < income1['total_income']:
        return 'низкий'
    elif income < income2['total_income']:
        return 'средний'
    else:
        return 'высокий'

debt_income['income_group'] = debt_income['total_income'].apply(income_group)
debt_income.head(10)

,total_income,debt,income_group
0,20667,1,низкий
1,21205,0,низкий
2,21367,0,низкий
3,21695,0,низкий
4,21895,0,низкий
5,22472,0,низкий
6,23844,0,низкий
7,24457,0,низкий
8,25227,0,низкий
9,25308,0,низкий


Сгруппируем таблицу debt_income по категориям income_group и подсчитаем количество взятых кредитов и неоплаченных кредитов
для каждой категории.

In [67]:
income = debt_income.groupby(['income_group']).agg({'debt':'count'})
income['fail'] = debt_income[debt_income['debt']==1].groupby(['income_group']).agg({'debt':'count'})
try:
    income['percent'] = income['fail'] / income['debt'] *100
except:
    print('warning: check income[]')
income.sort_values('percent')

,debt,fail,percent
income_group,,,
высокий,6451,482,7.471710
низкий,6450,524,8.124031
средний,6450,565,8.759690


### Вывод

Чаще остальных выплчаивают кредит вовремя люди с высоким уровнем дохода - всего 7.47% среди них не выплатили его вовремя. Хуже всего ситуация у среднего класса, среди них не выплатили кредит  8.75%.

- Как разные цели кредита влияют на его возврат в срок?

Создадим отдельную таблицу для удобства работы с уже сформированным столбцом purpose_group и debt

In [68]:
debt_purpose = df[['purpose_group', 'debt']]

In [69]:
purpose = debt_purpose.groupby(['purpose_group']).agg({'debt':'count'})
purpose['fail'] = debt_purpose[debt_purpose['debt']==1].groupby(['purpose_group']).agg({'debt':'count'})
try:
    purpose['percent'] = purpose['fail'] / purpose['debt'] *100
except:
    print('warning: check income[]')
purpose.sort_values('percent')

,debt,fail,percent
purpose_group,,,
недвижимость,10811,782,7.233373
свадьба,2324,186,8.003442
образование,4013,370,9.220035
автомобиль,4306,403,9.359034


### Вывод

Чаще остальных погашают кредит вовремя люди, оформившие его с целью покупки\аренды недвижимости - всего 7.23% из них не оплатили его вовремя. Хуже всего ситуация у людей оформивших кредит с целью связанной с автомобилем - не оплатили кредит вовремя 9.35% из них, что в 1.3 раза чаще.

### Шаг 4. Общий вывод

В ходе анализа предоставленных данных обнаружили, что люди без детей чаще оплачивают кредит вовремя, чем люди с детьми, причём чем больше детей, тем выше процент людей не оплативших кредит вовремя, а именно среди людей с 20 детьми оказались должниками 10.52% против 7.54% среди бездетных. Среди самой распространённой группы, с 1 ребёнком, должниками остались 9.23%. Однако люди с 3 детьми выбиваются из зависимости, среди них всего 8.18% должники. Возможно, дейтвительно люди с 3 детьми финансово стабильнее остальных семей.

Разделяя людей по семейному статусу обнаружили, что вдовы и вдовцы чаще остальных оплачивают кредит вовремя, среди них всего 6.56% остались должниками. Хуже всего ситуация у неженатых и незамужних, среди них не оплатили кредит вовремя 9.75%. Среди самой распространённой группы, женатые и замужние, не выплатили кредит 7.54%

Разделяя людей по доходу, обнаружим факт, что люди высокого дохода чаще остальных оплачивают кредит вовремя, 7.47% среди них остались должниками, при этом хуже всего ситуация у среднего класса - 8.75% отдали кредит вовремя. Мы не знаем суммы, которые брали в долг, но можно предположить, что дело в потребностях, люди среднего класса готовы потреблять больше необходимого стремясь к ощущению стабильности.

Разделяя людей по целям получения кредита обнаружили, что операции связанные с недвижимостью самые надежные, всего 7.23% среди них остались должниками, с автомобилями самые не надежные - 9.35% должников. Также можем обратить внимание на количество выданых кредитов по категориям обнаружили, что количество кредитов на операции с недвижимостью превышает сумму выданных кредитов по всем остальным категориям.

Итак, составляя портрет самого надежного заёмщика на основе полученных данных получим: вдова\вдовец без детей с высоким доходом, берущий кредит на операции с недвижимостью.
Самый ненадёжный заёмщик: неженатый\незамужняя с детьми, со средним доходом, берущий кредит на операции с автомобилем.